<a href="https://colab.research.google.com/github/Malik-Raheel/data_analysis/blob/main/KeyAssets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta
!pip install ccxt
!pip install pandas
!pip install numpy
!pip install ta
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=101247e835166a22ee4a7cbe39377781aead6e8c9565aebf82ab0fbaf53f79f9
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 25.6 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl s

In [2]:
import ccxt
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange

# Initialize the exchange
exchange = ccxt.kucoin()  # Replace 'kucoin' with your preferred exchange

# Symbols to analyze
symbols = ["BTC/USDT", "ETH/USDT", "AVAX/USDT", "SOL/USDT", "XAVA/USDT", "SLIM/USDT", "HNT/USDT", "NEAR/USDT",
           "DOGE/USDT", "SAND/USDT", "CRV/USDT", "CFX/USDT", "INJ/USDT", "LINK/USDT",
           "AAVE/USDT", "LTC/USDT", "ADA/USDT", "GAFI/USDT", "JUP/USDT", "MNT/USDT", "PYTH/USDT", "STG/USDT"]

# Function to fetch historical data
def fetch_ccxt_data(symbol, timeframe, limit=500):
    """Fetch historical OHLCV data using CCXT."""
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=["time", "open", "high", "low", "close", "volume"])
    df["time"] = pd.to_datetime(df["time"], unit="ms")
    return df

# Function to calculate indicators
def calculate_indicators(df, rsi_period=14, atr_period=14):
    """
    Calculate RSI and ATR indicators for the given dataframe.
    """
    df["RSI"] = RSIIndicator(df["close"], window=rsi_period).rsi()
    atr = AverageTrueRange(df["high"], df["low"], df["close"], window=atr_period)
    df["ATR"] = atr.average_true_range()
    return df

# Function to calculate PNL and trend
def calculate_pnl_and_trend(df):
    """
    Calculate the 30-day and 7-day PNL and determine the trend.
    """
    df["30d_pnl"] = ((df["close"].iloc[-1] - df["close"].iloc[-30]) / df["close"].iloc[-30]) * 100 if len(df) > 30 else None
    df["7d_pnl"] = ((df["close"].iloc[-1] - df["close"].iloc[-7]) / df["close"].iloc[-7]) * 100 if len(df) > 7 else None
    df["trend"] = "Upward" if df["30d_pnl"].iloc[-1] > 0 else ("Downward" if df["30d_pnl"].iloc[-1] < 0 else "Sideways")
    return df

# Function to generate trading signals with PNL and trend
def generate_signals(symbol, data_1d, data_4h, data_15m, data_weekly):
    """Generate buy/sell signals with PNL and trend data."""
    signals = []

    # Fetch PNL and trend data
    trend_30d = data_1d["trend"].iloc[-1]
    pnl_30d = data_1d["30d_pnl"].iloc[-1]
    pnl_7d = data_1d["7d_pnl"].iloc[-1]

    # Define risk-reward ratio
    risk_reward_ratio = 2

    # Daily chart (1D) signals
    if data_1d["RSI"].iloc[-1] < 40:
        price = data_1d["close"].iloc[-1]
        atr = data_1d["ATR"].iloc[-1]
        signals.append({
            "symbol": symbol,
            "timeframe": "1D",
            "action": "BUY",
            "price": price,
            "reason": f"Oversold on 1D RSI. 30D PNL: {pnl_30d:.2f}%, 7D PNL: {pnl_7d:.2f}%",
            "trend": trend_30d,
            "take_profit": price + risk_reward_ratio * atr,
            "stop_loss": price - atr
        })
    elif data_1d["RSI"].iloc[-1] > 80:
        price = data_1d["close"].iloc[-1]
        atr = data_1d["ATR"].iloc[-1]
        signals.append({
            "symbol": symbol,
            "timeframe": "1D",
            "action": "SELL",
            "price": price,
            "reason": f"Overbought on 1D RSI. 30D PNL: {pnl_30d:.2f}%, 7D PNL: {pnl_7d:.2f}%",
            "trend": trend_30d,
            "take_profit": price - risk_reward_ratio * atr,
            "stop_loss": price + atr
        })

    return signals

# Main execution
all_signals = []

for symbol in symbols:
    try:
        # Fetch data for each timeframe
        data_1d = fetch_ccxt_data(symbol, "1d")
        data_weekly = fetch_ccxt_data(symbol, "1w")

        # Calculate indicators and PNL
        data_1d = calculate_indicators(data_1d)
        data_weekly = calculate_indicators(data_weekly)
        data_1d = calculate_pnl_and_trend(data_1d)

        # Generate signals
        signals = generate_signals(symbol, data_1d, None, None, data_weekly)
        all_signals.extend(signals)

    except Exception as e:
        print(f"Error processing {symbol}: {e}")
        continue

# Output results
for signal in all_signals:
    print(f"{signal['symbol']} {signal['timeframe']} Signal: {signal['action']} at {signal['price']:.2f} "
          f"(TP: {signal['take_profit']:.2f}, SL: {signal['stop_loss']:.2f}, Reason: {signal['reason']}, Trend: {signal['trend']})")


Error processing VIRTUAL/USDT: index 13 is out of bounds for axis 0 with size 8
Error processing JOE/USDT: kucoin does not have market symbol JOE/USDT
HNT/USDT 1D Signal: BUY at 4.55 (TP: 5.79, SL: 3.94, Reason: Oversold on 1D RSI. 30D PNL: -31.83%, 7D PNL: -5.52%, Trend: Downward)
GAFI/USDT 1D Signal: BUY at 3.01 (TP: 3.81, SL: 2.61, Reason: Oversold on 1D RSI. 30D PNL: -9.59%, 7D PNL: -8.05%, Trend: Downward)
MNT/USDT 1D Signal: BUY at 1.01 (TP: 1.19, SL: 0.93, Reason: Oversold on 1D RSI. 30D PNL: -12.91%, 7D PNL: -8.28%, Trend: Downward)
